<a href="https://colab.research.google.com/github/towardsai/ragbook-notebooks/blob/main/notebooks/Chapter%2008%20-%20LangSmith_Introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q langchain==0.3.26 langchain-openai==0.3.23 langchain-community==0.3.26 langchain-core==0.3.66 openai==1.92.0 tiktoken==0.8.0 cohere==5.15.0 deeplake==4.2.11 langchain-deeplake==0.1.0 langchainhub==0.1.21

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 753.4/753.4 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.4/32.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source o

In [2]:
import os

# os.environ["OPENAI_API_KEY"] = "<YOUR_OPENAI_KEY>"
# os.environ["ACTIVELOOP_TOKEN"] = "<YOUR_ACTIVELOOP_KEY>"
# os.environ["LANGSMITH_API_KEY"] = "<YOUR_LANGSMITH_API_KEY"

from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["ACTIVELOOP_TOKEN"] = userdata.get('ACTIVELOOP_TOKEN')
os.environ["LANGSMITH_API_KEY"] = userdata.get('LangSmith_API_Key')
os.environ["USER_AGENT"] = "LangSmith-Introduction"

SecretNotFoundError: Secret OPENAI_API_KEY1 does not exist.

In [ ]:
# Load docs
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()
len( data )

In [ ]:
# Split
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 0)
all_splits = text_splitter.split_documents(data)
len( all_splits )

In [ ]:
# Store splits
from langchain_openai import OpenAIEmbeddings
from langchain_deeplake import DeeplakeVectorStore

my_activeloop_org_id = "" # TODO: use your organization id here
vectorstore = DeeplakeVectorStore.from_documents(
    all_splits,
    dataset_path=f"hub://{my_activeloop_org_id}/langsmith_intro",
    embedding=OpenAIEmbeddings(model="text-embedding-3-small"),
    overwrite=True
    )

In [ ]:
# LLM
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4.1-mini", temperature=0)

In [ ]:
from langsmith import Client
client = Client()
prompt = client.pull_prompt("rlm/rag-prompt")

print(prompt)

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langsmith import Client

retriever = vectorstore.as_retriever()

rag_chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
)

# Usage
question = "What are the approaches to Task Decomposition?"
result = rag_chain.invoke(question)
print("Answer:", result.content)